In [ ]:
import tensorflow as tf
import re
import os
import cv2

# Preprocessing label maps (.pbtxt)

In [12]:
def load_label_map(label_map_path, exclude_label="Lindsay-s Nail"):
    with open(label_map_path, "r") as file:
        content = file.read()

    label_map = {}
    matches = re.findall(r'name:\s*\"(.+?)\",\s*id:\s*(\d+),', content)  # Adjusted regex

    for match in matches:
        label_name = match[0]
        label_id = int(match[1])

        if label_name != exclude_label:  # Skip Lindsay's Nail because it only has 1 item
            label_map[label_id - 1] = label_name  # Convert to 0-based index

    return label_map

# Path to label map
label_map_path = "dataset/train/Diseases_label_map.pbtxt"
CLASS_LABELS = load_label_map(label_map_path)

# Debug: Print Loaded Labels
if CLASS_LABELS:
    print("✅ Loaded Labels (without Lindsay's Nail):", CLASS_LABELS)
else:
    print("❌ ERROR: No labels loaded. Check the pbtxt file path or format!")


✅ Loaded Labels (without Lindsay's Nail): {0: 'Acral Lentiginous Melanoma', 1: 'Beaus Line', 2: 'Blue Finger', 3: 'Clubbing', 4: 'Healthy Nail', 5: 'Koilonychia', 7: 'Muehrckes Lines', 8: 'Onychogryphosis', 9: 'Pitting', 10: 'Terry-s Nail'}


# Converting to Split Set (Test, Train, Validation)
This code converts the .tfrecords from the dataset into a Directory-Based Dataset which is split into the ``train``, ``test``, and ``valid`` sets. The output directory is at ``data``

In [ ]:
# Paths to datasets
tfrecord_paths = {
    "train": "misc/data_src/train/Diseases.tfrecord",
    "valid": "misc/data_src/valid/Diseases.tfrecord",
    "test": "misc/data_src/test/Diseases.tfrecord",
}

output_dirs = {
    "train": "data/train",
    "valid": "data/valid",
    "test": "data/test",
}

# Ensure output directories exist
for split, output_dir in output_dirs.items():
    os.makedirs(output_dir, exist_ok=True)

def _parse_tfrecord(example_proto):
    feature_description = {
        "image/encoded": tf.io.FixedLenFeature([], tf.string),
        "image/format": tf.io.FixedLenFeature([], tf.string),
        "image/object/class/label": tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)

    # Decode image
    image = tf.io.decode_jpeg(example["image/encoded"], channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.uint8).numpy()

    # Convert sparse tensor to dense
    labels = tf.sparse.to_dense(example["image/object/class/label"]).numpy()

    # Exclude missing labels
    if len(labels) > 0:
        label_id = labels[0] - 1  # Convert to 0-based index
        label_name = CLASS_LABELS.get(label_id)

        # Exclude images with missing or unknown labels
        if label_name is None:
            return None

        return image, label_name
    return None



# Process all datasets
for split, tfrecord_path in tfrecord_paths.items():
    output_dir = output_dirs[split]
    print(f"🔄 Processing {split} set...")

    raw_dataset = tf.data.TFRecordDataset([tfrecord_path])
    for i, raw_record in enumerate(raw_dataset):
        parsed = _parse_tfrecord(raw_record)
        if parsed:
            image, label_name = parsed
            class_dir = os.path.join(output_dir, label_name)
            os.makedirs(class_dir, exist_ok=True)
            filename = os.path.join(class_dir, f"{i}.jpg")
            cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Fix color issue

    print(f"✅ {split.capitalize()} set done! Images saved in: {output_dir}")

print("🎉 All datasets processed successfully!")

🔄 Processing train set...


2025-03-31 17:50:33.618918: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


✅ Train set done! Images saved in: converted_dataset/train
🔄 Processing valid set...
✅ Valid set done! Images saved in: converted_dataset/valid
🔄 Processing test set...
✅ Test set done! Images saved in: converted_dataset/test
🎉 All datasets processed successfully!


# Combined Set
This code converts the .tfrecords into one whole Directory-Based dataset. The output is at ``combined_dataset``
However, this is optional only, if we want a randomized training and validation set. We did not include this in our training.

In [16]:
# Paths to TFRecord files
tfrecord_paths = {
    "train": "dataset/train/Diseases.tfrecord",
    "valid": "dataset/valid/Diseases.tfrecord",
    "test": "dataset/test/Diseases.tfrecord",
}

# Output folder for FastAI
output_dir = "combined_dataset"
os.makedirs(output_dir, exist_ok=True)


# Process all datasets into one folder
for split, tfrecord_path in tfrecord_paths.items():
    print(f"🔄 Processing {split} set...")

    raw_dataset = tf.data.TFRecordDataset([tfrecord_path])
    for i, raw_record in enumerate(raw_dataset):
        parsed = _parse_tfrecord(raw_record)
        if parsed:
            image, label_name = parsed
            class_dir = os.path.join(output_dir, label_name)
            os.makedirs(class_dir, exist_ok=True)
            filename = os.path.join(class_dir, f"{split}_{i}.jpg")  # Prefix to avoid conflicts
            cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Fix color issue

    print(f"✅ {split.capitalize()} set done! Images saved in: {output_dir}")

print("🎉 All datasets processed successfully!")


🔄 Processing train set...
✅ Train set done! Images saved in: combined_dataset
🔄 Processing valid set...
✅ Valid set done! Images saved in: combined_dataset
🔄 Processing test set...
✅ Test set done! Images saved in: combined_dataset
🎉 All datasets processed successfully!
